# Fetch ERA5 data and make timeseries for stations

In [1]:
import NORA3_ERA5
import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import numpy as np

In [2]:
params = ["top_net_solar_radiation",
    "surface_net_solar_radiation",
    "surface_solar_radiation_downwards",
    "2m_temperature",
    "total_cloud_cover",
    "high_cloud_cover",
    "medium_cloud_cover",
    "low_cloud_cover",
    "snowfall",
    "total_precipitation",
    "snow_albedo",
    "forecast_albedo",
    "leaf_area_index_low_vegetation",
    "leaf_area_index_high_vegetation",
    "total_column_water_vapour"]

print(params)

nc_params = ["tsr",
    "ssr",
    "ssrd",
    "t2m",
    "tcc",
    "hcc",
    "mcc",
    "lcc",
    "sf",
    "tp",
    "asn",
    "fal",
    "lai_lv",
    "lai_hv",
    "tcwv"]

print(nc_params)

['top_net_solar_radiation', 'surface_net_solar_radiation', 'surface_solar_radiation_downwards', '2m_temperature', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', 'low_cloud_cover', 'snowfall', 'total_precipitation', 'snow_albedo', 'forecast_albedo', 'leaf_area_index_low_vegetation', 'leaf_area_index_high_vegetation', 'total_column_water_vapour']
['tsr', 'ssr', 'ssrd', 't2m', 'tcc', 'hcc', 'mcc', 'lcc', 'sf', 'tp', 'asn', 'fal', 'lai_lv', 'lai_hv', 'tcwv']


In [ ]:
%%time

for year in range(2015,2016):
    for param in params:
        filename = "ERA5_grid_{}_{}".format(year, param)
        NORA3_ERA5.retrieve_era5_data(filename, param, year)

In [6]:
stations = NORA3_ERA5.read_stations("stationlist.csv")

In [5]:
print(stations)

{'stationid': ['ÅS', 'DOVRE-LANNEM', 'HEMSEDAL SKISENTER', 'FLESLAND', 'BERGEN - FLORIDA UIB', 'TRONDHEIM - GLØSHAUGEN', 'KARASJOK - MARKANNJARGA', 'HOPEN', 'JAN MAYEN', 'GAUSDAL - FOLLEBU', 'JUVVASSHØE', 'RÅDE - TOMB', 'RYGGE - HUGGENES', 'LIER', 'HØNEFOSS - HVERVEN', 'GRAN', 'RAMNES - KILE VESTRE', 'TJØLLING', 'GJERPEN - ÅRHUS', 'BØ', 'ØSAKER', 'LYNGDAL', 'ETNE II', 'MIDTSTOVA', 'NJØS', 'LINGE', 'LEBERGSFJELLET', 'TINGVOLL', 'SURNADAL - SYLTE', 'SKJETLEIN', 'RISSA III', 'RENA - ØRNHAUGEN', 'VALNESFJORD', 'LOSISTUA', 'SORTLAND - KLEIVA', 'ALVDAL', 'SNØHEIM', 'ISKORAS II', 'FÅVANG', 'SANDE - GALLEBERG', 'RAKKESTAD', 'ÅRNES', 'ULLENSVANG FORSØKSGARD', 'ROVERUD', 'NORDNESFJELLET', 'OSLO - BLINDERN', 'PASVIK - SVANVIK', 'KVITHAMAR', 'ØSTRE TOTEN - APELSVOLL', 'LØKEN I VOLBU', 'LANDVIK', 'SÆRHEIM', 'FURENESET', 'TROMSØ - HOLT', 'KISE PA HEDMARK', 'Finse (UiO)', 'Sandefjord (Jotun)', 'Tromsø (UiT)', 'Galgu (UiT)'], 'longitude': [10.7818, 9.2143, 8.4968, 5.2265, 5.332, 10.4072, 25.5023, 25.0

In [4]:
def init_netcdf_output_file(out_da, station_ids, station_lons, station_lats):
    out_da["stationid"] = station_ids

    out_da["longitude_station"] = station_lons
    out_da["latitude_station"] = station_lats

    # ensure CF compliance
    out_da.attrs["Conventions"] = "CF-1.8"
    out_da.attrs["reference"] = "Hersbach et al. (2018): ERA5 hourly data on single levels from 1959 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). (Accessed on <18-01-2023 >), 10.24381/cds.adbb2d47"
    out_da.attrs["summary"] = "Timeseries extracted from ERA5 global reanalysis"
    out_da.attrs["project"] = "ERA5 and SUNPOINT"
    out_da.attrs["institute"] = "Norwegian Meteorological Institute"
    out_da.attrs["creator_url"] = "https://www.met.no"
    out_da.attrs["contact"] = "martinls@met.no"

    out_da["longitude"].attrs["units"] = "degrees_east"
    out_da["longitude"].attrs["long_name"] = "longitude"
    out_da["longitude"].attrs["description"] = "longitude of closest data point to station"

    out_da["longitude_station"].attrs["units"] = "degrees_east"
    out_da["longitude_station"].attrs["long_name"] = "longitude_station"

    out_da["latitude"].attrs["units"] = "degrees_north"
    out_da["latitude"].attrs["long_name"] = "latitude"
    out_da["latitude"].attrs["description"] = "latitude of closest data point to station"

    out_da["latitude_station"].attrs["units"] = "degrees_north"
    out_da["latitude_station"].attrs["long_name"] = "latitude_station"

In [7]:
%%time

raise

data_dir = "/lustre/storeB/users/martinls/src/MachineOcean-WP12/scripts/NORA3_ERA5/"

station_ids = stations["stationid"]
station_lons = stations["longitude"]
station_lats = stations["latitude"]

for (param, nc_param) in zip(params, nc_params):
    break # do not execute
    filename = data_dir + str(param) + ".nc"
    print("Writing timeseries for {}".format(param))
    era5 = xr.open_dataset(filename)
    out_da = xr.Dataset()

    era5_da = era5[nc_param].sel(longitude=xr.DataArray(station_lons, dims="station"), latitude=xr.DataArray(station_lats, dims="station"), method="nearest")
    era5_da.attrs["coordinates"] = "longitude latitude"
    out_da[param] = era5_da
    
    init_netcdf_output_file(out_da, station_ids, station_lons, station_lats)
    # We are not getting these variables from an existing nc-file, and therefore need to 
    # ensure that the correct dimension (station) is used
    out_da["stationid"] = out_da["stationid"].swap_dims({"stationid": "station"})
    #out_da["longitude"] = out_da["longitude"].expand_dims(dim="station")
    out_da["longitude_station"] = out_da["longitude_station"].swap_dims({"longitude_station": "station"})
    #out_da["latitude"] = out_da["latitude"].expand_dims(dim="station")
    out_da["latitude_station"] = out_da["latitude_station"].swap_dims({"latitude_station": "station"})
    out_da.to_netcdf(str(param) + "_ts.nc", 
                                format="NETCDF4", engine="netcdf4", unlimited_dims="time", mode="w",
                                encoding={param: {"dtype": "float32", "zlib": False, "_FillValue": 1.0e37}})

Writing timeseries for leaf_area_index_high_vegetation
CPU times: user 8.3 s, sys: 5min 10s, total: 5min 18s
Wall time: 9min 12s


In [12]:
raise

import xarray as xr

nc_files = [param + "_ts.nc" for param in params]

all_timeseries = xr.open_mfdataset(nc_files, parallel=True, combine="nested",
                  data_vars='minimal', coords='minimal', compat='override')

all_timeseries.to_netcdf("ERA5_ts.nc", format="NETCDF4", engine="netcdf4", unlimited_dims="time", mode="w")

/modules/rhel8/conda/install/envs/production-10-2022/lib/python3.9/site-packages/xarray/conventions.py:205: SerializationWarning: variable stationid has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


In [6]:
raise

# concat all params year for year
import xarray as xr

years = list(range(1991, 2016))

for year in years:
    nc_files = ["ERA5_grid_" + str(year) + "_" + param + ".nc" for param in params]

    all_timeseries = xr.open_mfdataset(nc_files, parallel=True, combine="nested",
                      data_vars='minimal', coords='minimal', compat='override')

    all_timeseries.to_netcdf("ERA5_grid_" + str(year) + ".nc", format="NETCDF4", engine="netcdf4", unlimited_dims="time", mode="w")

In [11]:
#raise

# concat periods of multiple years from single-year files
import xarray as xr

periods = [list(range(1996, 2001)),
           list(range(2001, 2006)),
           list(range(2006, 2011)),
           list(range(2011, 2016)),
          ]

for years in periods:
    nc_files = ["ERA5_grid_" + str(year) + ".nc" for year in years]
    
    all_timeseries = xr.open_mfdataset(nc_files, parallel=True, combine="nested",
                      data_vars='minimal', coords='minimal', compat='override')
    
    all_timeseries.to_netcdf("ERA5_grid_" + str(years[0]) + "_" + str(years[-1]) + ".nc", format="NETCDF4", engine="netcdf4", unlimited_dims="time", mode="w")

/modules/rhel8/conda/install/envs/production-10-2022/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/modules/rhel8/conda/install/envs/production-10-2022/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     arra